In [1]:
%load_ext kedro.ipython

[08/15/24 23:38:39] INFO     Using                                                                  ]8;id=65412;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=469542;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/framework/project/__init__.py#246\246]8;;\
                             '/home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/p                
                             ython3.11/site-packages/kedro/framework/project/rich_logging.yml' as                  
                             logging configuration.                                                                

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=784221;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=679098;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py#58\58]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=217570;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=900392;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py#60\60]8;;\

                    INFO     Resolved project path as: /mnt/c/Users/Winson Yeap/Documents/0         ]8;id=652464;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=536656;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py#175\175]8;;\
                             Python/Kaggle/bank-loan-defaulter-prediction.                                         
                             To set a different path, run '%reload_kedro <project_root>'                           

[08/15/24 23:38:40] INFO     Kedro project Bank Loan Defaulter Prediction                           ]8;id=504377;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=304024;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=149101;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=665039;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py#142\142]8;;\
                             'pipelines'                                                                           

[08/15/24 23:38:41] INFO     Registered line magic 'run_viz'                                        ]8;id=981723;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=312163;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/ipython/__init__.py#148\148]8;;\

In [46]:
import logging

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

_logger = logging.getLogger(__name__)
_logger.setLevel(logging.DEBUG)

In [3]:
train_df = catalog.load("feature_engineered_train_dataset")
test_df = catalog.load("feature_engineered_test_dataset")

[08/15/24 23:38:46] INFO     Loading data from feature_engineered_train_dataset                 ]8;id=313709;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=324385;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/io/data_catalog.py#537\537]8;;\
                             (FeatherDataset)...                                                                   

                    INFO     Loading data from feature_engineered_test_dataset                  ]8;id=148002;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=407705;file:///home/winsonyeap/miniconda3/envs/bank-loan-defaulter-prediction/lib/python3.11/site-packages/kedro/io/data_catalog.py#537\537]8;;\
                             (FeatherDataset)...                                                                   

### Settings & Parameters

In [7]:
CATEGORICAL_VARS = [
    'GRADE', 'SUB_GRADE', 'HOME_OWNERSHIP', 'VERIFICATION_STATUS', 'LOAN_TITLE', 'INITIAL_LIST_STATUS', 'APPLICATION_TYPE', 
    'IS_CONSOLIDATION',
]

## Statistical Test
---
### Chi-Squared Test of Independence

In [5]:
from scipy.stats import chi2_contingency

In [55]:
chi2_results = dict()
for var in CATEGORICAL_VARS:
    contingency_table = pd.crosstab(train_df[var], train_df['LOAN_STATUS'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape) - 1
    cramers_v = np.sqrt(chi2 / (n * min_dim))
    chi2_results[var] = {'P_VALUE': p_value, 'CRAMERS_V': cramers_v}
chi2_results = pd.DataFrame(chi2_results).T.reset_index(drop=False).rename(columns={'index': 'FEATURE'}).sort_values(by='P_VALUE', ascending=True)

In [58]:
chi2_results.style.background_gradient(cmap='BuGn_r', vmin=0, vmax=1)

,FEATURE,P_VALUE,CRAMERS_V
2,HOME_OWNERSHIP,0.000319,0.015449
5,INITIAL_LIST_STATUS,0.000504,0.013393
4,LOAN_TITLE,0.003142,0.030678
0,GRADE,0.029361,0.014418
7,IS_CONSOLIDATION,0.029854,0.008362
1,SUB_GRADE,0.505268,0.022193
3,VERIFICATION_STATUS,0.718691,0.003129
6,APPLICATION_TYPE,1.000000,0.000000


In [25]:
pd.crosstab(train_df['HOME_OWNERSHIP'], train_df['LOAN_STATUS'], normalize='index').style.background_gradient(cmap='BuGn', axis=None)

LOAN_STATUS,0,1
HOME_OWNERSHIP,,
MORTGAGE,0.911337,0.088663
OWN,0.898449,0.101551
RENT,0.904306,0.095694


In [26]:
pd.crosstab(train_df['INITIAL_LIST_STATUS'], train_df['LOAN_STATUS'], normalize='index').style.background_gradient(cmap='BuGn', axis=None)

LOAN_STATUS,0,1
INITIAL_LIST_STATUS,,
FORWARDED,0.903286,0.096714
WAITING,0.911099,0.088901


In [27]:
pd.crosstab(train_df['LOAN_TITLE'], train_df['LOAN_STATUS'], normalize='index').style.background_gradient(cmap='BuGn', axis=None)

LOAN_STATUS,0,1
LOAN_TITLE,,
BILL CONSOLIDATION,0.933333,0.066667
BILLS,0.687500,0.312500
BUSINESS,0.923497,0.076503
CAR FINANCING,0.868056,0.131944
CONSOLIDATION,0.911765,0.088235
CREDIT CARD,0.888889,0.111111
CREDIT CARD CONSOLIDATION,0.885714,0.114286
CREDIT CARD DEBT,0.888889,0.111111
CREDIT CARD LOAN,0.733333,0.266667


In [28]:
pd.crosstab(train_df['GRADE'], train_df['LOAN_STATUS'], normalize='index').style.background_gradient(cmap='BuGn', axis=None)

LOAN_STATUS,0,1
GRADE,,
A,0.908752,0.091248
B,0.912763,0.087237
C,0.906104,0.093896
D,0.903620,0.096380
E,0.904127,0.095873
F,0.896260,0.103740
G,0.893651,0.106349


In [29]:
pd.crosstab(train_df['SUB_GRADE'], train_df['LOAN_STATUS'], normalize='index').style.background_gradient(cmap='BuGn', axis=None)

LOAN_STATUS,0,1
SUB_GRADE,,
A1,0.908358,0.091642
A2,0.902014,0.097986
A3,0.905638,0.094362
A4,0.908569,0.091431
A5,0.905932,0.094068
B1,0.900137,0.099863
B2,0.911080,0.088920
B3,0.917479,0.082521
B4,0.908113,0.091887


In [30]:
pd.crosstab(train_df['IS_CONSOLIDATION'], train_df['LOAN_STATUS'], normalize='index').style.background_gradient(cmap='BuGn', axis=None)

LOAN_STATUS,0,1
IS_CONSOLIDATION,,
0,0.905359,0.094641
1,0.910280,0.089720


In [32]:
pd.crosstab(train_df['VERIFICATION_STATUS'], train_df['LOAN_STATUS'], normalize='index').style.background_gradient(cmap='BuGn', axis=None)

LOAN_STATUS,0,1
VERIFICATION_STATUS,,
Not Verified,0.907884,0.092116
Source Verified,0.906617,0.093383
Verified,0.908729,0.091271


In [33]:
pd.crosstab(train_df['APPLICATION_TYPE'], train_df['LOAN_STATUS'], normalize='index').style.background_gradient(cmap='BuGn', axis=None)

LOAN_STATUS,0,1
APPLICATION_TYPE,,
INDIVIDUAL,0.907484,0.092516
JOINT,0.910569,0.089431
